In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess

class BGGSpider(scrapy.Spider):
    name = "BGGS"
    
    # Here is where we insert our API call.
    start_urls = [
        'https://boardgamegeek.com/browse/boardgame/page/1'
        ]
            
    def parse(self, response):
        
        rows = response.xpath('//table[@id="collectionitems"]/tr')[1:]
        
        for row in rows:
            
            # Yield a dictionary with the values we want.
            yield {
                'rank': row.xpath('td[@class="collection_rank"]/a/@name').extract_first(),
                'thumbnail': row.xpath('td[@class="collection_thumbnail"]/a/img/@src').extract_first(),
                'title': row.xpath('td[starts-with(@class,"collection_objectname")]/div/a/text()').extract_first(),
                'geekrating': row.xpath('td[@class="collection_bggrating"][1]/text()').extract_first().strip(),
                'avgrating': row.xpath('td[@class="collection_bggrating"][2]/text()').extract_first().strip(),
                'numvoters': row.xpath('td[@class="collection_bggrating"][3]/text()').extract_first().strip()
                # 'listprice': row.xpath('td[@class="collection_shop"]/div/div/div').extract()
                # list price might not be scrape-able? can't find it
            }
        # Get the URL of the previous page.
        next_page = response.xpath('//a[starts-with(@href, "/browse/boardgame/page/")]/@href').extract_first()
        
        # There are a LOT of pages here.  For our example, we'll just scrape the first 9.
        # This finds the page number. The next segment of code prevents us from going beyond page 9.
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        # Recursively call the spider to run on the next page, if it exists.
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            yield scrapy.Request(next_page, callback=self.parse)
            
    
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'PythonLinks.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    # We use CLOSESPIDER_PAGECOUNT to limit our scraper to the first 100 links.    
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

# Starting the crawler with our spider.
process.crawl(WikiSpider)
process.start()
print('First 100 links extracted!')

ModuleNotFoundError: No module named 'scrapy'